In [1]:
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn import datasets
from sklearn.base import ClassifierMixin
from sklearn.datasets import fetch_mldata, fetch_20newsgroups

from sklearn.neighbors.base import NeighborsBase, KNeighborsMixin, SupervisedIntegerMixin 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier,KDTree
from sklearn.model_selection import KFold

from sklearn.metrics.pairwise import pairwise_distances
from scipy.sparse.csr import csr_matrix

from math import sqrt
#%load_ext pycodestyle_magic

In [16]:
#%%pycodestyle

class MyKNeighborsClassifier(NeighborsBase, KNeighborsMixin, SupervisedIntegerMixin, ClassifierMixin):
    
    def __init__(self, n_neighbors, algorithm='brute'):
        self.n_neighbors=n_neighbors
        self.alg=(algorithm=='brute')
    
    def fit(self, X, y):
        self.y=y
        if self.alg:
            if type(X)==csr_matrix:
                self.x=X
            else:
                self.x=X.T[np.newaxis,...]
        else:
            self.tree=KDTree(X, leaf_size=40)
    
    def predict(self, X,metric='euclidean'):
        if self.alg:
            if type(X)==csr_matrix:
                dist=pairwise_distances(X,self.x,metric=metric)
            else:
                dist=np.sqrt(((X[...,np.newaxis]-self.x)**2).sum(axis=1))
            #dist=scipy.spatial.distance.cdist(X,self.x)
            tmp=np.argpartition(dist, self.n_neighbors,axis=-1)[:,:self.n_neighbors]
        else:
            tmp = self.tree.query(X, k=self.n_neighbors,return_distance =False)
        tmp=self.y[tmp][...,np.newaxis]
        labels=np.unique(self.y)[np.newaxis][np.newaxis]
        return labels[0,0,(labels==tmp).sum(axis=1).argmax(axis=1)]
            
    
    def predict_proba(self, X,metric='euclidean'):
        if self.alg:
            if type(X)==csr_matrix:
                dist=pairwise_distances(X,self.x,metric=metric)
            else:
                dist=np.sqrt(((X[...,np.newaxis]-self.x)**2).sum(axis=1))
            #dist=scipy.spatial.distance.cdist(X,self.x)
            tmp=np.argpartition(dist, self.n_neighbors,axis=-1)[:,:self.n_neighbors]
        else:  
            tmp = self.tree.query(X, k=self.n_neighbors,return_distance =False)
            
        tmp=self.y[tmp][...,np.newaxis]
        labels=np.unique(self.y)[np.newaxis][np.newaxis]
        tmp= (labels==tmp).sum(axis=1)
        return tmp/tmp.sum(axis=1)[:,np.newaxis]
    
    def score(self, X, y,metric='euclidean'):
        return (self.predict(X,metric)==y).sum()/len(y)

In [17]:
help(pairwise_distances)

Help on function pairwise_distances in module sklearn.metrics.pairwise:

pairwise_distances(X, Y=None, metric='euclidean', n_jobs=None, **kwds)
    Compute the distance matrix from a vector array X and optional Y.
    
    This method takes either a vector array or a distance matrix, and returns
    a distance matrix. If the input is a vector array, the distances are
    computed. If the input is a distances matrix, it is returned instead.
    
    This method provides a safe way to take a distance matrix as input, while
    preserving compatibility with many other algorithms that take a vector
    array.
    
    If Y is given (default is None), then the returned matrix is the pairwise
    distance between the arrays from both X and Y.
    
    Valid values for metric are:
    
    - From scikit-learn: ['cityblock', 'cosine', 'euclidean', 'l1', 'l2',
      'manhattan']. These metrics support sparse matrix inputs.
    
    - From scipy.spatial.distance: ['braycurtis', 'canberra', 'cheb

In [18]:
iris = datasets.load_iris()


In [19]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)


In [20]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='brute')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='brute')

In [21]:
clf.fit(X_train, y_train)
my_clf.fit(X_train, y_train)

In [22]:
assert abs(my_clf.score(X_test, y_test) - clf.score(X_test,y_test))<0.005, "Score must be simillar"

In [23]:
print(clf.score(X_test,y_test))
print(my_clf.score(X_test,y_test))

0.8
0.8


In [24]:
%time clf.fit(X_train, y_train)

Wall time: 0 ns


KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=2, p=2,
           weights='uniform')

In [25]:
%time my_clf.fit(X_train, y_train)

Wall time: 0 ns


In [26]:
%time clf.predict(X_test)

Wall time: 1.95 ms


array([0, 1, 2, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 2])

In [27]:
%time my_clf.predict(X_test)

Wall time: 1.24 ms


array([0, 1, 2, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 2])

In [28]:
%time clf.predict_proba(X_test)

Wall time: 1.79 ms


array([[1. , 0. , 0. ],
       [0. , 1. , 0. ],
       [0. , 0. , 1. ],
       [1. , 0. , 0. ],
       [1. , 0. , 0. ],
       [0. , 1. , 0. ],
       [0. , 1. , 0. ],
       [0. , 0.5, 0.5],
       [0. , 1. , 0. ],
       [0. , 1. , 0. ],
       [1. , 0. , 0. ],
       [0. , 1. , 0. ],
       [1. , 0. , 0. ],
       [0. , 1. , 0. ],
       [0. , 0. , 1. ]])

In [29]:
%time my_clf.predict_proba(X_test)

Wall time: 994 µs


array([[1. , 0. , 0. ],
       [0. , 1. , 0. ],
       [0. , 0. , 1. ],
       [1. , 0. , 0. ],
       [1. , 0. , 0. ],
       [0. , 1. , 0. ],
       [0. , 1. , 0. ],
       [0. , 0.5, 0.5],
       [0. , 1. , 0. ],
       [0. , 1. , 0. ],
       [1. , 0. , 0. ],
       [0. , 1. , 0. ],
       [1. , 0. , 0. ],
       [0. , 1. , 0. ],
       [0. , 0. , 1. ]])

In [30]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')

In [31]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [32]:
%time clf.fit(X_train, y_train)

Wall time: 39.1 ms


KNeighborsClassifier(algorithm='kd_tree', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=2, p=2,
           weights='uniform')

In [33]:
%time my_clf.fit(X_train, y_train)

Wall time: 1e+03 µs


In [34]:
%time clf.predict(X_test)

Wall time: 18.9 ms


array([2, 2, 1, 1, 0, 2, 0, 1, 0, 2, 2, 1, 1, 0, 0])

In [35]:
%time my_clf.predict(X_test)

Wall time: 0 ns


array([2, 2, 1, 1, 0, 2, 0, 1, 0, 2, 2, 1, 1, 0, 0])

In [36]:
%time clf.predict_proba(X_test)

Wall time: 998 µs


array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [37]:
%time my_clf.predict_proba(X_test)

Wall time: 1.03 ms


array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [38]:
assert abs(my_clf.score(X_test, y_test) - clf.score(X_test,y_test))<0.005, "Score must be simillar"

In [39]:
newsgroups = fetch_20newsgroups(subset='train',remove=['headers','footers', 'quotes'])

In [40]:
data = newsgroups['data']
target = newsgroups['target']

In [41]:
data_tok=[]
indices=[]
for ind,line in enumerate(data):
    line=line.lower()
    tmp=list(line)
    for i, c in enumerate(line):
        if(not(c.isalpha() or c.isdigit())):
            tmp[i]=' '
    tmp=''.join(tmp).split()
    if tmp:
        data_tok.append(tmp)
        indices.append(ind)
target=target[indices]

In [42]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower() , map(' '.join, data_tok))), "please make sure that you lowercase the data and drop spaced texts"

In [43]:
from scipy.sparse import csr_matrix
indptr = [0]
indices = []
data = []
words = {}
for d in data_tok:
    data+=[1]*len(d)
    for token in d:
        index = words.setdefault(token, len(words))
        indices.append(index)
    indptr.append(len(indices))

sm=csr_matrix((data, indices, indptr), dtype=int)

In [44]:
n_splits=3
kf = KFold(n_splits=n_splits,shuffle=False)
best_k=0
best_score=-1.0
for k in range(1,11):
    score_sum=0
    my_clf = MyKNeighborsClassifier(n_neighbors=k, algorithm='brute')
    for train_indx,test_indx in kf.split(sm):
        my_clf.fit(sm[train_indx],target[train_indx])
        score_sum+=my_clf.score(sm[test_indx],target[test_indx])
    if score_sum/n_splits>best_score:
        best_score=score_sum/n_splits
        best_k=k
    print(score_sum/n_splits)
print('best k =',best_k)

0.21628498727735368
0.1931115957833515
0.18738640494365685
0.1933842239185751
0.19665576154125772
0.1975645219920029
0.1991094147582697
0.20219920029080332
0.20456197746274082
0.20619774627408216
best k = 1


In [45]:
n_splits=3
kf = KFold(n_splits=n_splits,shuffle=False)
best_k=0
best_score=-1.0
for k in range(1,11):
    score_sum=0
    my_clf = MyKNeighborsClassifier(n_neighbors=k, algorithm='brute')
    for train_indx,test_indx in kf.split(sm):
        my_clf.fit(sm[train_indx],target[train_indx])
        score_sum+=my_clf.score(sm[test_indx],target[test_indx],'cosine')
    if score_sum/n_splits>best_score:
        best_score=score_sum/n_splits
        best_k=k
    print(score_sum/n_splits)
print('best k =',best_k)

0.28789531079607417
0.2728098873137041
0.27162849872773537
0.26472191930207195
0.2604507451835696
0.2585423482370047
0.2584514721919302
0.2611777535441658
0.2579062159214831
0.25808796801163214
best k = 1


In [70]:
n_splits=3
kf = KFold(n_splits=n_splits,shuffle=False)
best_k=0
best_score=-1.0
for k in range(1,11):
    score_sum=0
    my_clf = KNeighborsClassifier(n_neighbors=k, algorithm='brute')
    for train_indx,test_indx in kf.split(sm):
        my_clf.fit(sm[train_indx],target[train_indx])
        score_sum+=my_clf.score(sm[test_indx],target[test_indx])
    if score_sum/n_splits>best_score:
        best_score=score_sum/n_splits
        best_k=k
    print(score_sum/n_splits)
print('best k =',best_k)

0.21628498727735368
0.1931115957833515
0.1887495456197746
0.1933842239185751
0.19665576154125772
0.1975645219920029
0.1991094147582697
0.20219920029080332
0.20456197746274082
0.20619774627408216
best k = 1


In [552]:
docs = [["hello", "hello", "hello"], ["goodbye", "cruel", "world"],["goodbye", "cruel", "world"]]
indptr = [0]
indices = []
data = []
vocabulary = {}
for d in docs:
    for term in d:
        index = vocabulary.setdefault(term, len(vocabulary))
        indices.append(index)
        data.append(1)
    indptr.append(len(indices))

sm1=csr_matrix((data, indices, indptr), dtype=int)

In [553]:
docs = [["hello", "world", "hell"],['goodbye','hell','hell']]
indptr = [0]
indices = []
data = []
vocabulary = {}
for d in docs:
    for term in d:
        index = vocabulary.setdefault(term, len(vocabulary))
        indices.append(index)
        data.append(1)
    indptr.append(len(indices))

sm2=csr_matrix((data, indices, indptr), dtype=int)

In [556]:
import sklearn.metrics.pairwise
d=sklearn.metrics.pairwise.pairwise_distances(sm1,sm2)

In [557]:
d

array([[2.44948974, 3.74165739],
       [1.41421356, 1.41421356],
       [1.41421356, 1.41421356]])

In [338]:
help(csr_matrix)

Help on class csr_matrix in module scipy.sparse.csr:

class csr_matrix(scipy.sparse.compressed._cs_matrix, scipy.sparse.sputils.IndexMixin)
 |  csr_matrix(arg1, shape=None, dtype=None, copy=False)
 |  
 |  Compressed Sparse Row matrix
 |  
 |  This can be instantiated in several ways:
 |      csr_matrix(D)
 |          with a dense matrix or rank-2 ndarray D
 |  
 |      csr_matrix(S)
 |          with another sparse matrix S (equivalent to S.tocsr())
 |  
 |      csr_matrix((M, N), [dtype])
 |          to construct an empty matrix with shape (M, N)
 |          dtype is optional, defaulting to dtype='d'.
 |  
 |      csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])
 |          where ``data``, ``row_ind`` and ``col_ind`` satisfy the
 |          relationship ``a[row_ind[k], col_ind[k]] = data[k]``.
 |  
 |      csr_matrix((data, indices, indptr), [shape=(M, N)])
 |          is the standard CSR representation where the column indices for
 |          row i are stored in ``indices[indpt

In [342]:
import scipy
help(scipy.spatial.distance.cdist)

Help on function cdist in module scipy.spatial.distance:

cdist(XA, XB, metric='euclidean', *args, **kwargs)
    Compute distance between each pair of the two collections of inputs.
    
    See Notes for common calling conventions.
    
    Parameters
    ----------
    XA : ndarray
        An :math:`m_A` by :math:`n` array of :math:`m_A`
        original observations in an :math:`n`-dimensional space.
        Inputs are converted to float type.
    XB : ndarray
        An :math:`m_B` by :math:`n` array of :math:`m_B`
        original observations in an :math:`n`-dimensional space.
        Inputs are converted to float type.
    metric : str or callable, optional
        The distance metric to use.  If a string, the distance function can be
        'braycurtis', 'canberra', 'chebyshev', 'cityblock', 'correlation',
        'cosine', 'dice', 'euclidean', 'hamming', 'jaccard', 'jensenshannon',
        'kulsinski', 'mahalanobis', 'matching', 'minkowski', 'rogerstanimoto',
        'russell